In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px
from opencage.geocoder import OpenCageGeocode

In [2]:
tempByCity = pd.read_csv(r'C:\Users\sangi\Desktop\Info progetto python\Datasets\GlobalLandTemperaturesByCity.csv').dropna().reset_index(drop = True)
#tempByCountry = pd.read_csv(r'C:\Users\sangi\Desktop\Info progetto python\Datasets\GlobalLandTemperaturesByCountry.csv')
tempByMajorCity = pd.read_csv(r'C:\Users\sangi\Desktop\Info progetto python\Datasets\GlobalLandTemperaturesByMajorCity.csv')
#tempByState = pd.read_csv(r'C:\Users\sangi\Desktop\Info progetto python\Datasets\GlobalLandTemperaturesByState.csv')

In [3]:
cities = tempByCity[["City", "Country", "Latitude", "Longitude"]].groupby(["City"]).first().reset_index()

In [4]:
majorCities = tempByMajorCity[["City", "Country", "Latitude", "Longitude"]].groupby(["City"]).first().reset_index()

In [7]:
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3448 entries, 0 to 3447
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   City       3448 non-null   object
 1   Country    3448 non-null   object
 2   Latitude   3448 non-null   object
 3   Longitude  3448 non-null   object
dtypes: object(4)
memory usage: 107.9+ KB


In [5]:
def conversion():
    key = "1867ae77e86948d9a54c0a7bb9f3d0bd"
    geocoder = OpenCageGeocode(key)
    list_lat = []
    list_lon = []
    for index, row in majorCities.iterrows():
        City = row["City"]
        Country = row["Country"]
        query = str(City) + ',' + str(Country)
        results = geocoder.geocode(query)
        lat = results[0]['geometry']['lat']
        lon = results[0]['geometry']['lng']
        list_lat.append(lat)
        list_lon.append(lon)
    majorCities['Latitude'] = list_lat
    majorCities['Longitude'] = list_lon

conversion()

RateLimitExceededError: Your rate limit has expired. It will reset to 2500 on 2023-10-31T00:00:00

In [ ]:
cities.head()

In [ ]:
#gpd.points_from_xy(citiesLat, citiesLon,)

I need to convert the latitude and the longitude into a more convenient format, so that I can plot it more easily. I'll convert from DMS (degrees, minutes and seconds) to DD (decimal degrees).
I will convert the North and East values into positive values, and the South and West values into negative values.

In [ ]:
#fig = px.scatter_geo(tempByMajorCity, lat = tempByMajorCity["Latitude"], lon = tempByMajorCity["Longitude"], hover_name = tempByMajorCity["City"])
#fig.show()

In [ ]:
majorCities.info()